<a href="https://colab.research.google.com/github/NoelRodriguezR/Data-Analysis-Tutorial/blob/main/Customer_Segmentation_RFM_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [5]:
# Load the dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"

df = pd.read_excel(url)

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [34]:
#Check the shapre of the dataset
print(df.shape)

# Check for missing values
print(df.isnull().sum())

# Check column data types
print(df.dtypes)

(397924, 8)
InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64
InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object


In [35]:
# Drop rows with missing CustomerID
df = df.dropna(subset=['CustomerID'])

# Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Remove canceled orders (negative quantities)
df = df[df['Quantity'] > 0]

In [39]:
# Calculate Recency, Frequency, and Monetary values
import datetime as dt

# Set a reference date (e.g., the day after the last transaction)
snapshot_date = df['InvoiceDate'].max() + dt.timedelta(days=1)

# Calculate RFM metrics
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',  # Frequency
    'UnitPrice': 'sum'  # Monetary
})

# Rename columns
rfm.rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalPrice': 'UnitPrice'
}, inplace=True)

# Display the first 5 rows of the RFM table
rfm.head()

,Recency,Frequency,UnitPrice
CustomerID,,,
12346.0,326,1,1.04
12347.0,2,7,481.21
12348.0,75,4,178.71
12349.0,19,1,605.10
12350.0,310,1,65.30


In [44]:
# Create RFM quartiles
rfm['R_Quartile'] = pd.qcut(rfm['Recency'], 4, labels=[4, 3, 2, 1])
rfm['F_Quartile'] = pd.qcut(rfm['Frequency'], 4, labels=[1, 2, 3, 4])
rfm['M_Quartile'] = pd.qcut(rfm['Monetary'], 4, labels=[1, 2, 3, 4])

# Combine RFM scores
rfm['RFM_Score'] = rfm['R_Quartile'].astype(str) + rfm['F_Quartile'].astype(str) + rfm['M_Quartile'].astype(str)

# Display the first 5 rows of the RFM table with scores
rfm.head()

ValueError: Bin edges must be unique: Index([1.0, 1.0, 2.0, 5.0, 210.0], dtype='float64', name='Frequency').
You can drop duplicate edges by setting the 'duplicates' kwarg

In [45]:
# Define segments
segmentation_map = {
    r'[4][4-4][4-4]': 'Best Customers',
    r'[3-4][3-4][3-4]': 'Loyal Customers',
    r'[2-3][2-3][2-3]': 'Potential Loyalists',
    r'[1-2][1-2][1-2]': 'At-Risk Customers',
    r'[1][1-2][1-2]': 'Lost Customers'
}

# Assign segments
rfm['Segment'] = rfm['RFM_Score'].replace(segmentation_map, regex=True)

# Count customers in each segment
segment_counts = rfm['Segment'].value_counts()

# Plot the results
plt.figure(figsize=(10, 6))
segment_counts.plot(kind='bar', color='skyblue')
plt.title('Customer Segments')
plt.xlabel('Segment')
plt.ylabel('Number of Customers')
plt.show()

KeyError: 'RFM_Score'

In [46]:
# Plot Recency vs Monetary
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Recency', y='Monetary', hue='Segment', data=rfm, palette='viridis')
plt.title('Recency vs Monetary by Segment')
plt.xlabel('Recency (Days)')
plt.ylabel('Monetary ($)')
plt.show()

ValueError: Could not interpret value `Monetary` for `y`. An entry with this name does not appear in `data`.

<Figure size 1000x600 with 0 Axes>